In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [14]:
i = 0
inputs = []
targets = []
sp_text = ""
for line in open('sp4.txt'):
        line = line.strip()
        tar = line[1:]
        tar.strip()
        inp = line[:-1]
        inp.strip()
        inputs.append(inp)
        targets.append(tar)
        sp_text += line.strip("\n")
        i = i+1
        if i>=500: #00:
            break
print(tar)

utters	That's it! Eric, hurry! Somethin's goin' on out here!


In [26]:
vocab = list(set(sp_text))

vocab_stoi = {s: i for i, s in enumerate(vocab)}
vocab_itos = {i: s for i, s in enumerate(vocab)}
len(vocab)

70

In [27]:
def text_to_tensor(text, vocab=vocab):
    """Return a tensor containing the indices of characters in `text`."""
    indices = [vocab_stoi[ch] for ch in text]
    return torch.tensor(indices)

In [28]:
class SouthParkGenerator(nn.Module):
    def __init__(self, vocab_size, hidden_size, n_layers=1):
        super(SouthParkGenerator, self).__init__()
        # RNN attributes
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        # identiy matrix for generating one-hot vectors
        self.ident = torch.eye(vocab_size)
        # recurrent neural network
        self.rnn = nn.RNN(vocab_size, hidden_size, n_layers, batch_first=True)
        # a fully-connect layer that decodes the RNN output to
        # a distribution over the vocabulary
        self.decoder = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, inp, hidden):
        # reshape the input tensor to [1, seq_length]
        inp = inp.view(1, -1)
        # generate one-hot vectors from token indices
        inp = self.ident[inp]
        # obtain the next output and hidden state
        output, hidden = self.rnn(inp, hidden)
        # run the decoder
        output = self.decoder(output.squeeze(0))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(self.n_layers, 1, self.hidden_size)
    
model = SouthParkGenerator(len(vocab), 128)

In [29]:
hidden = model.init_hidden()

inp = inputs[0]
tar = targets[0]

print(len(inp))
print(len(tar))

inp = text_to_tensor(inp)
tar = text_to_tensor(tar)


output,_ = model(inp,hidden)
print(output.shape)
print(hidden.shape)

43
43
torch.Size([43, 70])
torch.Size([1, 1, 128])


In [30]:
def evaluate(model, prime_str='stan', predict_len=100, temperature=0.8):
    hidden = model.init_hidden()
    prime_input = text_to_tensor(prime_str)
    predicted = prime_str
    
    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = vocab_itos[top_i]
        predicted += predicted_char
        inp = text_to_tensor(predicted_char)

    return predicted

print(evaluate(model, predict_len=20))

stanhhb0Ln?qBhs?1:4Nhq8,


In [40]:
def train(model, inputs, targets, num_iters=200, lr=0.004):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    for it in range(num_iters):
        for i in range(len(inputs)):
            # get training set
            inp = inputs[i]
            tar = targets[i]
            inp = text_to_tensor(inp)
            tar = text_to_tensor(tar)
            # cleanup
            optimizer.zero_grad()
            # forward pass
            hidden = model.init_hidden()
            output, _ = model(inp, hidden)
            loss = criterion(output, tar)
            # backward pass
            loss.backward()
            optimizer.step()

            if i % 200 == 199:
                print("[Iter %d] Loss %f" % (i+1, float(loss)))
                print("    " + evaluate(model, ' ', 50))

train(model,inputs,targets)

[Iter 200] Loss 2.165726
     jore love anond	Yeis you ho th. Bhes gosr	cher wry
[Iter 400] Loss 2.492105
     mull	Nil onou t ancarsis I ybed oonr dype mankimpa
[Iter 200] Loss 2.042012
     toy Come hey	(aven	Gopor bayleringolyeanomenye	Che
[Iter 400] Loss 2.389437
     cart ag	rello. Come be ploplo nownptoploplo se peo
[Iter 200] Loss 2.121476
     Mamb you hohe boys! Dhyse you youfha cayll ane dok
[Iter 400] Loss 2.428221
     ou, Ikid mous mom as, reand goo Be perand mo plon'
[Iter 200] Loss 1.915737
     tfarens! Chef!) hit no thile lan't boinghe waylu o
[Iter 400] Loss 2.269169
     and, pat?! seeple. Come on tyan ming tuaks gutp ba
[Iter 200] Loss 1.772491
     juendeme on deet he dot chit! Good-bythe! he's the
[Iter 400] Loss 2.324997
     wankerigod mald restrizso hen wis weyle god deve t
[Iter 200] Loss 1.720736
     Cart and Bue what you down! Bhing??) ould. Chef!) 
[Iter 400] Loss 2.204195
     tongete peeplo gotth Pe pand thome all bett p on, 
[Iter 200] Loss 1.673331
   

[Iter 400] Loss 2.636855
     MarrehWer gh. Hwy goue, dee we s	m Iont! k	't He s
[Iter 200] Loss 2.398949
     Mun	Ghes Chels, ch! ythinoris aithily holy Is! Che
[Iter 400] Loss 2.635875
     Mal	Co. Como bourytis aroldy	Wher Thofls	Yourrinnk
[Iter 200] Loss 2.392158
     Mon	Broll Cive ris we n! tongharefle! it! m! Chit'
[Iter 400] Loss 2.634193
     Mun	Wto dlle risn is, bovignch, yoll thrt ris lyby
[Iter 200] Loss 2.386705
     Mac	Hofly to! I'se thha! Cou tf? to Gase his Chinn
[Iter 400] Loss 2.633691
     Mursa he Chatureer his Cherens idrinker Cher wh!om
[Iter 200] Loss 2.381421
     Mac	Oh! I' onole-! hing ha sh! yyfyber thae he y u
[Iter 400] Loss 2.633504
     Man	res Perveto The tybP, raous. I'ks sore s.. I't
[Iter 200] Loss 2.375740
     Muriemye  le's! I''nke sh wis. Iaebevh! me bouhe'h
[Iter 400] Loss 2.634510
     Mnl	Thas iks iver yor go Her we Jhnd tow kn's Then
[Iter 200] Loss 2.370845
     Morse lo! ke! r! ean! hon! th! et hh! I rigay! I't
[Iter 400] Loss 2.635207
   

[Iter 200] Loss 2.021889
     Mon	Whiths! Chi! Byeere ror-s	DoC th! If! Co the C
[Iter 400] Loss 2.619902
     Mon	Oh, Younid he I mow'slle ar younonvidit go Dad
[Iter 200] Loss 2.005249
     Mon	Oh!)ort! Ca to this chat youne yfwe go!) ll eo
[Iter 400] Loss 2.597981
     Mon	The's. Youre sy. Hevike me. Cer,'s, I errit yi
[Iter 200] Loss 1.988949
     Mon	But's bame outo bally. it's wheherere't btving
[Iter 400] Loss 2.572952
     Mocr	That? Iw! Is	C. Sar yous Wea whadoncereis ger
[Iter 200] Loss 1.960445
     Mon	Oh to yeay! Byon Chil hishan eo loul barke bef
[Iter 400] Loss 2.540034
     Mon	Nove onon! tor heveng s nay rights of?? AThTHA
[Iter 200] Loss 1.949441
     Mon	O!!)hat?! Bull ris ao's he you wit's, gone out
[Iter 400] Loss 2.506310
     Moringnnws, lomer cor oon torer on, fo bow's rig t
[Iter 200] Loss 1.927328
     Con kse aye  Chef! I goly there se- hoCly! Chennd 
[Iter 400] Loss 2.475496
     Mon	Wit is, Cheering.. Thild or fow ber sore sor g
[Iter 200] Loss 1.903064
   

[Iter 400] Loss 2.042830
     Caan	THAT is I's my is. Hey inme on, me with"., Cl
[Iter 200] Loss 1.853759
     Mavinn do or mire think hetay it Chef nood-bodld I
[Iter 400] Loss 2.081470
     dave Chef, ruome byee it peo! He mo suever	Aly peo
[Iter 200] Loss 1.532486
     Mavente realas anay. I'm you not yous whe nooll ri
[Iter 400] Loss 2.053027
     daveon't thybentnghte. That's for my ma to make ke
[Iter 200] Loss 1.602039
     four nood-boy I beirn nood, to willizze ho! Horve 
[Iter 400] Loss 2.075870
     Man	Wrat	WiF! Think here	Yeah? And.. CClers. I'mer
[Iter 200] Loss 1.563773
     Coly	Oh! He bye he' thing a have come chou ay! Uhe
[Iter 400] Loss 1.963889
     Man	THolarving all You'l to Tal on't sorro he's lo
[Iter 200] Loss 1.651115
     Bonnghiballo hake! You going ifuy! Come ond right 
[Iter 400] Loss 1.931897
     Mald witals Thef	Taver pons lcF's orr br right. I 
[Iter 200] Loss 1.536472
     Goven Childrienn kell right you like hell. uen ay,
[Iter 400] Loss 2.079613
   